In [1]:
import evaluate
from datasets import *
import numpy as np
import pathlib as pl
import pandas as pd
import torch
import ast
import os
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever, TransformersReader, Seq2SeqGenerator
from haystack.nodes.answer_generator.transformers import _BartEli5Converter
from haystack.pipelines import Pipeline

YEAR = 2022

MODEL_PATH_B = "../../data/models/BERT"
MODEL_PATH_T = "../../data/models/T5"

In [2]:
written_data = load_dataset('csv', data_files=f"../../data/clean/sr-2022-questions-answers-ALL-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=42)
written_data["test"] = written_data["test"].map(
    lambda example: ast.literal_eval(example["answers"]))

written_data["test"] = written_data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                  "text": example["text"], "answer_start": example["answer_start"]}})
written_data["test"] = written_data["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
written_data["test"].remove_columns(["text", "answer_start"])

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-54c2d14257a19ede/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-54c2d14257a19ede\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3ff29462153ffd09.arrow and C:\Users\rjutr\.cache\huggingface\datasets\csv\default-54c2d14257a19ede\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8202b31bf7f0cc8f.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-54c2d14257a19ede\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-68763aea809abd97.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-54c2d14257a19ede\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-50d87829045139cf.arrow
Loading cached process

Dataset({
    features: ['question', 'context', 'answers'],
    num_rows: 21
})

In [3]:
document_store = FAISSDocumentStore.load(index_path="document_store.faiss", config_path="document_store.json")

In [4]:
retriever = DensePassageRetriever.load(load_dir=f"../../data/models/DPR/{YEAR}", document_store=document_store, use_gpu=True)

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# Set the device
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define the prediction function
def inference_answer(model, question):
    question = question
    with torch.no_grad():
        outputs = model.run(query=question, params={"Model": {"top_k": 1}, "Retriever": {"top_k": 1}})
    return outputs["answers"][0].answer, outputs["documents"][0].content

In [6]:
results = {
    "Model": [],
    "Question": [],
    "Ground Truth Context": [],
    "Ground Truth Answer": [],
    "Retrieved Context": [],
    "Extracted/Generated Answer": []
}

In [7]:
reader = TransformersReader(model_name_or_path=f"{MODEL_PATH_B}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2022-handwritten", use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=reader, name="Model", inputs=["Retriever"])

for idx in range(written_data["test"].shape[0]):
    results["Model"].append("distilbert-base-handwritten")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del reader
del pipe
torch.cuda.empty_cache()

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [8]:
reader = TransformersReader(model_name_or_path=f"{MODEL_PATH_B}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2042-full_combined", use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=reader, name="Model", inputs=["Retriever"])

for idx in range(written_data["test"].shape[0]):
    results["Model"].append("distilbert-base-full_combined")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del reader
del pipe
torch.cuda.empty_cache()

In [9]:
reader = TransformersReader(model_name_or_path=f"{MODEL_PATH_B}/roberta-base-squad2-finetuned-NLB-QA-2022-handwritten", use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=reader, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("roberta-base-handwritten")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del reader
del pipe
torch.cuda.empty_cache()

In [10]:
reader = TransformersReader(model_name_or_path=f"{MODEL_PATH_B}/roberta-base-squad2-finetuned-NLB-QA-2042-full_combined", use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=reader, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("roberta-base-full_combined")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del reader
del pipe
torch.cuda.empty_cache()

In [11]:
generator = Seq2SeqGenerator(model_name_or_path=f"{MODEL_PATH_T}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten", input_converter=_BartEli5Converter(), use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=generator, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("t5-base-handwritten")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del generator
del pipe
torch.cuda.empty_cache()

In [12]:
generator = Seq2SeqGenerator(model_name_or_path=f"{MODEL_PATH_T}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten", input_converter=_BartEli5Converter(), use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=generator, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("t5-small-handwritten")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del generator
del pipe
torch.cuda.empty_cache()

In [13]:
generator = Seq2SeqGenerator(model_name_or_path=f"{MODEL_PATH_T}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined", input_converter=_BartEli5Converter(), use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=generator, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("t5-base-full_combined")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del generator
del pipe
torch.cuda.empty_cache()

In [14]:
generator = Seq2SeqGenerator(model_name_or_path=f"{MODEL_PATH_T}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined", input_converter=_BartEli5Converter(), use_gpu=True)
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=generator, name="Model", inputs=["Retriever"])



for idx in range(written_data["test"].shape[0]):
    results["Model"].append("t5-small-full_combined")
    row = written_data["test"][idx]
    results["Question"].append(row["question"].strip())
    results["Ground Truth Context"].append(row["context"])
    results["Ground Truth Answer"].append(row["answers"]["text"][0])
    gotten_answer, gotten_context = inference_answer(pipe, row["question"].strip())
    results["Retrieved Context"].append(gotten_context)
    results["Extracted/Generated Answer"].append(gotten_answer)

del generator
del pipe
torch.cuda.empty_cache()

In [15]:
results_df = pd.DataFrame(results)

In [16]:
def get_answers_for_question(question):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(results_df[results_df["Question"] == question])

In [17]:
get_answers_for_question("Why was EU Taxonomy developed?")

,Model,Question,Ground Truth Context,Ground Truth Answer,Retrieved Context,Extracted/Generated Answer
0,distilbert-base-handwritten,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",The EU taxonomy is linked to the EU’s broader policy\nobjectives
21,distilbert-base-full_combined,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",help to achieve a more sustainable and equitable future
42,roberta-base-handwritten,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",By aligning\neconomic activities with these objectives
63,roberta-base-full_combined,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",linked to the EU’s broader policy\nobjectives
84,t5-base-handwritten,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...","By aligning economic activities with these objectives, the EU taxonomy can h..."
105,t5-small-handwritten,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...","aligning economic activities with these objectives, the EU taxonomy can help..."
126,t5-base-full_combined,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",the EU’s broader policy objectives
147,t5-small-full_combined,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",linked to the EU’s broader policy objectives


In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results_df)

,Model,Question,Ground Truth Context,Ground Truth Answer,Retrieved Context,Extracted/Generated Answer
0,distilbert-base-handwritten,Why was EU Taxonomy developed?,To achieve ambitious goal of net-zero Green House Gas\r (GHG) emissions by 2...,To achieve ambitious goal of net-zero Green House Gas (GHG) emissions by 205...,"The EU taxonomy is linked to the EU’s broader policy\nobjectives, such as th...",The EU taxonomy is linked to the EU’s broader policy\nobjectives
1,distilbert-base-handwritten,How many employees are in Montenegro according to the CBCR report for 2022?,Number of employees Revenues from thirdparty sales Revenues from intra-grou...,380,››› Original verification certificate is in the Appendix 6 of this report.\n...,3
2,distilbert-base-handwritten,Which is the relevant KPI for impact area Inclusive & healthy economies?,Number of active digital users \r in retail (*an active digital user \r is a...,Number of active digital users in retail,"Out those, three areas were prioritised:\n2. Resource efficiency & security\...","Sustainability Report 2021, p.12"
3,distilbert-base-handwritten,What does the National action plan for respecting human rights in economy fo...,In June 2022 the NLB d.d. decided on joining the commitment to the Slovenian...,"Precarious work, gender equality, working environment for people with disabi...","Out those, three areas were prioritised:\n2. Resource efficiency & security\...",priorities of impact areas
4,distilbert-base-handwritten,Where can we find more information about NLB Group whistle-blowing system?,"For more information about our whistle-blowing system, refer to: https://whi...",On https://whistler.nlb.si/faq-eng.,"Moreover,\ncommunication with the whistle-blower is carried out inside\nthe ...",communication with the whistle-blower is carried out inside\nthe application
5,distilbert-base-handwritten,How many whistle-blow cases were reported at the NLB Group level in 2022?,"Among them, 10 reports of suspicions of harmful behaviour were categorised a...",10.,"Moreover,\ncommunication with the whistle-blower is carried out inside\nthe ...",41
6,distilbert-base-handwritten,What is respect for human rights an indicator of?,Respect for human rights is an indicator of the maturity\r of a social and b...,Respect for human rights is an indicator of the maturity of a social and bus...,The Code applies to all employees in NLB and in the\nmembers of the NLB Grou...,the maturity\nof a social and business environment
7,distilbert-base-handwritten,What will the Supervisory Board of NLB annualy assess with regard to its ind...,"As a rule, the Supervisory Board of NLB shall once a year\r assess its compo...","the Supervisory Board of NLB shall once a year assess its composition, perfo...",It also reflects the standards that are expected relationship\nwith the Bank...,stakeholders
8,distilbert-base-handwritten,What represents Bankarium?,"Together with Bankarium, which presents the history \r of Slovenian banking ...","Bankarium presents the history of Slovenian banking industry, development o...",The management body as a whole must cover an adequately wide range of knowle...,Continuity of composition of the management\nbody and senior management
9,distilbert-base-handwritten,How many Conflict of Interest cases were processed by the Members of the NLB...,Total cases processed Whistleblow cases* Anti-Bribery and Corruption Conflic...,1,Activities and results in 2022\nNLB Group continued its focus on fulfilling ...,13


In [19]:
# split the dataset bz model
for model in results_df["Model"].unique():
    model_df = results_df[results_df["Model"] == model]
    model_df.to_csv(f"../../data/results/2022-handwritten/{model}-outputs.csv", index=False, sep=";")